In [1]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY")    
)

### Setting tools

In [10]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

In [ ]:
import arxiv # arxiv.org (archive of articles)

def article_search(title: str):
    search = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    results = [
        f"Title: {article.title}\n"
        f"Category: {article.primary_category}\n"
        f"Link: {article.entry_id}\n"
        for article in search.results()
    ]

    return f"\n\n".join(results)

In [5]:
article_search_tool = FunctionTool.from_defaults(fn=article_search)

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[article_search_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)

response = agent.chat(
    "Me retorne artigos sobre langchain na educação"
)

### With Tavily

In [20]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=os.environ.get("TAVILY_API_KEY")
)

In [21]:
tavily_tool_list = tavily_tool.to_tool_list()

for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [22]:
tavily_tool.search("Me retorne artigos cientificos sobre LangChain", max_results=3)

[Document(id_='d7a8999c-e23b-41d8-bbd7-e1c50a6fb00a', embedding=None, metadata={'url': 'https://medium.com/fretebras-tech/langchain-construindo-um-chatpdf-com-rag-retrieval-augmented-generation-openai-e-streamlit-61125e2122de'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Por conta desses acontecimentos, fiquei por um tempo afastado dos meus estudos. Mas por meio desse artigo compartilho meu retorno e comprometimento com os estudos, que sempre gosto de compartilhar por aqui, por meio de artigos, um pouco sobre eles.\n\nObrigado pela sua atenção! E até breve 😄\n\n--\n\n--\n\n1\n\nPublished in Fretebras Tech [...] LangChain é uma biblioteca open-source que tem como objetivo facilitar a criação de aplicações utilizando LLMs, ou seja, modelos de linguagem.\n\nDessa forma, o LangChain disponibiliza diversos tipos de componentes e fu

In [ ]:
tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily search",
    description="Busca artigos com Tavily sobre um determinado tópico"
)